## Normalized market with sizing effect

In [1]:
from model_builder import model_builder
from data_gen import simulate_private_equity_cashflows
import numpy as np

In [2]:
cashflows = simulate_private_equity_cashflows()

In [3]:
# Normalize the cashflows of each fund in the market

def fund_weights(cashflows):
    funds = cashflows['FundID'].unique() 
    weights = {}
    for fund in funds:
        fund_cashflows = cashflows[cashflows['FundID'] == fund]
        weights[fund] = -fund_cashflows['Cashflow'].sum()
    return weights

cashflows_contr = cashflows[cashflows['Cashflow'] < 0]
eq_cashflows_contr = fund_weights(cashflows_contr)

def eq_calc(row):
    fund = row['FundID']
    cf = row['Cashflow']
    
    return cf / eq_cashflows_contr[fund]

# Apply the function row-wise
cashflows['eq_cashflow'] = cashflows.apply(eq_calc, axis=1)

# Drop and rename
cashflows = cashflows.drop(columns=['Cashflow']).rename(columns={'eq_cashflow': 'Cashflow'})

cashflows

,FundID,VintageYear,Strategy,Geography,FundQuality,Quarter,date,Cashflow
0,0,2003,Venture,Rest of World,Bad,0,2003-03-31,-0.110380
1,0,2003,Venture,Rest of World,Bad,1,2003-06-30,-0.070752
2,0,2003,Venture,Rest of World,Bad,2,2003-09-30,-0.064530
3,0,2003,Venture,Rest of World,Bad,3,2003-12-31,-0.073506
4,0,2003,Venture,Rest of World,Bad,4,2004-03-31,-0.056660
...,...,...,...,...,...,...,...,...
119064,2399,1996,Buyout,Rest of World,Bad,43,2006-12-31,0.052958
119065,2399,1996,Buyout,Rest of World,Bad,44,2007-03-31,0.045725
119066,2399,1996,Buyout,Rest of World,Bad,45,2007-06-30,0.000000
119067,2399,1996,Buyout,Rest of World,Bad,46,2007-09-30,0.015371


In [4]:
def portfolio_cashflows(cashflows, portfolio_size):
    # Randomly select fund IDs
    selected_funds = np.random.choice(cashflows['FundID'].unique(), size=portfolio_size, replace=False)
    
    # Create a random size adjustment for each selected fund
    size_adjustments = {fund: np.random.uniform(0.5, 2.0) for fund in selected_funds}
    
    # Filter the portfolio
    portfolio = cashflows[cashflows['FundID'].isin(selected_funds)].copy()
    
    # Apply the size adjustment
    portfolio['Cashflow'] = portfolio.apply(lambda row: row['Cashflow'] * size_adjustments[row['FundID']], axis=1)
    
    return portfolio

In [5]:
port_cfs = portfolio_cashflows(cashflows, 200)
port_cfs = port_cfs.copy()
model = model_builder(cashflows, port_cfs)

### Monte-carlo simulation

In [32]:
market_model = model_builder(cashflows, cashflows)
market_model

({'Portfolio MOIC': 1.9132413771615897,
  'Market MOIC': 1.9132413771615897,
  'Timing Alpha MOIC': -2.220446049250313e-16,
  'Strategy Alpha MOIC': -2.220446049250313e-16,
  'Geography Alpha MOIC': -2.220446049250313e-16,
  'Sizing Alpha MOIC': 0.0,
  'Residual Alpha MOIC': 6.661338147750939e-16},
 {'Portfolio IRR': 0.11354391650455908,
  'Market IRR': 0.11354391650455908,
  'Timing Alpha IRR': 1.942890293094024e-16,
  'Strategy Alpha IRR': 1.942890293094024e-16,
  'Geography Alpha IRR': 1.942890293094024e-16,
  'Sizing Alpha IRR': -2.3592239273284576e-16,
  'Residual Alpha IRR': -3.469446951953614e-16})

In [72]:
number_of_simulations = 100
fund_size = 200
moic_results = []
irr_results = []

for i in range(number_of_simulations):
    port_cfs = portfolio_cashflows(cashflows, fund_size)
    port_cfs = port_cfs.copy()
    model = model_builder(cashflows, port_cfs)
    moic_results.append(model[0])
    irr_results.append(model[1])


In [43]:
def average_dicts(dict_list):
    """Compute the average of a list of dictionaries."""
    keys = dict_list[0].keys()
    avg_dict = {}
    for key in keys:
        avg_dict[key] = sum(d[key] for d in dict_list) / len(dict_list)
    return avg_dict

In [73]:
avg_moic_decomposition = average_dicts(moic_results)
avg_irr_decomposition = average_dicts(irr_results)

In [74]:
avg_irr_decomposition

{'Portfolio IRR': 0.11318978211728598,
 'Market IRR': 0.11354391650455925,
 'Timing Alpha IRR': 0.00018783212993540881,
 'Strategy Alpha IRR': 0.00014638805183436951,
 'Geography Alpha IRR': 0.00027898456397964263,
 'Sizing Alpha IRR': 2.37388865306494e-05,
 'Residual Alpha IRR': -0.0009910780195531584}

In [75]:
avg_moic_decomposition

{'Portfolio MOIC': 1.9103922912086282,
 'Market MOIC': 1.9132413771615902,
 'Timing Alpha MOIC': 0.0005033111993123218,
 'Strategy Alpha MOIC': 0.0011209679924466265,
 'Geography Alpha MOIC': 0.0027676333575359304,
 'Sizing Alpha MOIC': -6.146126142814179e-05,
 'Residual Alpha MOIC': -0.00717953724082798}

## Analyze the individual effect

### Strategy effect

In [76]:
venture_ids = cashflows[cashflows['Strategy'] == 'Venture']['FundID'].unique()
buyout_ids = cashflows[cashflows['Strategy'] == 'Buyout']['FundID'].unique()
venture_ids = np.append(venture_ids,buyout_ids[0]) # we need at least one venture fund to make the model work
buyout_ids = np.append(buyout_ids,venture_ids[0]) # we need at least one buyout fund to make the model work

In [77]:
def portfolio_cashflows_by_id(cashflows, ids):

    # Create a random size adjustment for each selected fund
    size_adjustments = {fund: np.random.uniform(0.5, 2.0) for fund in ids}
    
    # Filter the portfolio
    portfolio = cashflows[cashflows['FundID'].isin(ids)].copy()
    
    # Apply the size adjustment
    portfolio['Cashflow'] = portfolio.apply(lambda row: row['Cashflow'] * size_adjustments[row['FundID']], axis=1)
    
    return portfolio

In [78]:
port_cfs_venture = portfolio_cashflows_by_id(cashflows, venture_ids)
port_cfs_venture = port_cfs_venture.copy()
port_cfs_buyout = portfolio_cashflows_by_id(cashflows, buyout_ids)
port_cfs_buyout = port_cfs_buyout.copy()

In [79]:
model_venture = model_builder(cashflows, port_cfs_venture)
model_venture

({'Portfolio MOIC': 2.182694667466335,
  'Market MOIC': 1.9132413771615897,
  'Timing Alpha MOIC': 0.001628138871754281,
  'Strategy Alpha MOIC': 0.2670068795619307,
  'Geography Alpha MOIC': -0.009885606645638756,
  'Sizing Alpha MOIC': 0.004990763061409886,
  'Residual Alpha MOIC': 0.005713115455288964},
 {'Portfolio IRR': 0.14145156880180076,
  'Market IRR': 0.11354391650455908,
  'Timing Alpha IRR': 0.0002700161823088326,
  'Strategy Alpha IRR': 0.02774126660040188,
  'Geography Alpha IRR': 0.00038001128426620734,
  'Sizing Alpha IRR': 0.00024941322853963555,
  'Residual Alpha IRR': -0.0007330549982748763})

In [80]:
model_buyout = model_builder(cashflows, port_cfs_buyout)
model_buyout

({'Portfolio MOIC': 1.6701870739211384,
  'Market MOIC': 1.9132413771615897,
  'Timing Alpha MOIC': -0.0012259638666529682,
  'Strategy Alpha MOIC': -0.25543572770086254,
  'Geography Alpha MOIC': 0.01153559303764995,
  'Sizing Alpha MOIC': 0.011333220582565584,
  'Residual Alpha MOIC': -0.009261425293151326},
 {'Portfolio IRR': 0.0864890090206433,
  'Market IRR': 0.11354391650455908,
  'Timing Alpha IRR': -8.530975224463333e-06,
  'Strategy Alpha IRR': -0.028784752788373447,
  'Geography Alpha IRR': 0.00011860078101326443,
  'Sizing Alpha IRR': 0.0016776261822776944,
  'Residual Alpha IRR': -5.7850683608834386e-05})

### Geo effect

In [81]:
america_ids = cashflows[cashflows['Geography'] == 'North America']['FundID'].unique()
row_ids = cashflows[cashflows['Geography'] == 'Rest of World']['FundID'].unique()
america_ids = np.append(america_ids,row_ids[0]) # we need at least one venture fund to make the model work
row_ids = np.append(row_ids,america_ids[0]) # we need at least one buyout fund to make the model work

In [82]:
port_cfs_america = portfolio_cashflows_by_id(cashflows, america_ids)
port_cfs_america = port_cfs_america.copy()
port_cfs_row = portfolio_cashflows_by_id(cashflows, row_ids)
port_cfs_row = port_cfs_row.copy()

In [83]:
model_america = model_builder(cashflows, port_cfs_america)
model_america

({'Portfolio MOIC': 2.3281147035357606,
  'Market MOIC': 1.9132413771615897,
  'Timing Alpha MOIC': 0.0012476371201057823,
  'Strategy Alpha MOIC': -0.011983275975596897,
  'Geography Alpha MOIC': 0.39732148944794154,
  'Sizing Alpha MOIC': 0.02354561164867741,
  'Residual Alpha MOIC': 0.004741864133043006},
 {'Portfolio IRR': 0.14890664947135837,
  'Market IRR': 0.11354391650455908,
  'Timing Alpha IRR': 1.1344790237982627e-05,
  'Strategy Alpha IRR': -0.0009702677700760276,
  'Geography Alpha IRR': 0.03565678873856806,
  'Sizing Alpha IRR': 0.00029206153932306855,
  'Residual Alpha IRR': 0.00037280566874620114})

In [84]:
model_row = model_builder(cashflows, port_cfs_row)
model_row

({'Portfolio MOIC': 1.469307765435622,
  'Market MOIC': 1.9132413771615897,
  'Timing Alpha MOIC': 0.0021840576731415595,
  'Strategy Alpha MOIC': 0.006538658346052051,
  'Geography Alpha MOIC': -0.43487005875458107,
  'Sizing Alpha MOIC': -0.017528885344679335,
  'Residual Alpha MOIC': -0.00025738364590099927},
 {'Portfolio IRR': 0.06496134276347316,
  'Market IRR': 0.11354391650455908,
  'Timing Alpha IRR': 0.0003538597541422883,
  'Strategy Alpha IRR': -0.00018808154074874406,
  'Geography Alpha IRR': -0.04797672446436543,
  'Sizing Alpha IRR': -0.0018707761160409453,
  'Residual Alpha IRR': 0.0010991486259269156})

### Fund quality

In [85]:
good_ids = cashflows[cashflows['FundQuality'] == 'Good']['FundID'].unique()
bad_ids = cashflows[cashflows['FundQuality'] == 'Bad']['FundID'].unique()
good_ids = np.append(good_ids,bad_ids[0]) # we need at least one venture fund to make the model work
bad_ids = np.append(bad_ids,good_ids[0]) # we need at least one buyout fund to make the model work

In [86]:
port_cfs_good = portfolio_cashflows_by_id(cashflows, good_ids)
port_cfs_good = port_cfs_good.copy()    
port_cfs_bad = portfolio_cashflows_by_id(cashflows, bad_ids)
port_cfs_bad = port_cfs_bad.copy()

In [87]:
model_good = model_builder(cashflows, port_cfs_good)
model_good

({'Portfolio MOIC': 2.4717859149163393,
  'Market MOIC': 1.9132413771615897,
  'Timing Alpha MOIC': -0.0032170406945657426,
  'Strategy Alpha MOIC': 0.012909529497238115,
  'Geography Alpha MOIC': 0.004251617461964008,
  'Sizing Alpha MOIC': -0.02062987723638443,
  'Residual Alpha MOIC': 0.5652303087264976},
 {'Portfolio IRR': 0.16254577935064426,
  'Market IRR': 0.11354391650455908,
  'Timing Alpha IRR': -0.00015171791123982925,
  'Strategy Alpha IRR': 0.0015979067450611895,
  'Geography Alpha IRR': 3.0054044007243874e-05,
  'Sizing Alpha IRR': -0.0031149344164498327,
  'Residual Alpha IRR': 0.05064055438470641})

In [88]:
model_bad = model_builder(cashflows, port_cfs_bad)
model_bad

({'Portfolio MOIC': 1.321878717142031,
  'Market MOIC': 1.9132413771615897,
  'Timing Alpha MOIC': 0.0016577876181085305,
  'Strategy Alpha MOIC': -0.014899739043762406,
  'Geography Alpha MOIC': -0.00939604829679963,
  'Sizing Alpha MOIC': 0.0021293236990558295,
  'Residual Alpha MOIC': -0.570853983996161},
 {'Portfolio IRR': 0.04562917162090492,
  'Market IRR': 0.11354391650455908,
  'Timing Alpha IRR': -0.00028031859133119397,
  'Strategy Alpha IRR': -0.0017015141166047826,
  'Geography Alpha IRR': -0.0005806413153480461,
  'Sizing Alpha IRR': -0.0001690758636035128,
  'Residual Alpha IRR': -0.06518319499676663})

### Vintage effect

In [89]:
vintage_ids = {}
for vint in cashflows['VintageYear'].unique():
    vint_ids = cashflows[cashflows['VintageYear'] == vint]['FundID'].unique()
    vintage_ids[vint] = vint_ids

In [90]:
import random
def pick_one_per_vintage(fund_dict):
    selected_ids = []

    for vintage, fund_ids in fund_dict.items():
        # Randomly pick one fund from this vintage
        fund_ids = list(fund_ids)  # ensure it's a list
        selected_id = random.choice(fund_ids)
        selected_ids.append(selected_id)

    return np.array(selected_ids)

one_from_each = pick_one_per_vintage(vintage_ids)

In [91]:
good_vintage_ids = cashflows[cashflows['VintageYear'] % 2 == 1]['FundID'].unique()
bad_vintage_ids = cashflows[cashflows['VintageYear'] % 2 == 0]['FundID'].unique()
good_vintage_ids = np.append(good_vintage_ids,one_from_each) # we need at least one venture fund to make the model work
bad_vintage_ids = np.append(bad_vintage_ids,one_from_each) # we need at least one buyout fund to make the model work

In [92]:
port_cfs_good_vintage = portfolio_cashflows_by_id(cashflows, good_vintage_ids)
port_cfs_good_vintage = port_cfs_good_vintage.copy()
port_cfs_bad_vintage = portfolio_cashflows_by_id(cashflows, bad_vintage_ids)
port_cfs_bad_vintage = port_cfs_bad_vintage.copy()

In [93]:
model_good_vintage = model_builder(cashflows, port_cfs_good_vintage)
model_good_vintage

({'Portfolio MOIC': 2.1589791422277567,
  'Market MOIC': 1.9132413771615897,
  'Timing Alpha MOIC': 0.2390477303915084,
  'Strategy Alpha MOIC': 0.005064207334172455,
  'Geography Alpha MOIC': 0.032582595908614786,
  'Sizing Alpha MOIC': 0.005314218255803915,
  'Residual Alpha MOIC': -0.036270986823932594},
 {'Portfolio IRR': 0.14185781662921462,
  'Market IRR': 0.11354391650455908,
  'Timing Alpha IRR': 0.026745503745859323,
  'Strategy Alpha IRR': 0.0015587535733163627,
  'Geography Alpha IRR': 0.005000572469897574,
  'Sizing Alpha IRR': 0.001311429725767227,
  'Residual Alpha IRR': -0.006302359390184947})

In [94]:
model_bad_vintage = model_builder(cashflows, port_cfs_bad_vintage)
model_bad_vintage

({'Portfolio MOIC': 1.6747949824005188,
  'Market MOIC': 1.9132413771615897,
  'Timing Alpha MOIC': -0.2473412127318122,
  'Strategy Alpha MOIC': 0.0696383291230207,
  'Geography Alpha MOIC': 0.0053152703295475945,
  'Sizing Alpha MOIC': 0.006543027420515157,
  'Residual Alpha MOIC': -0.07260180890234214},
 {'Portfolio IRR': 0.08693360626430874,
  'Market IRR': 0.11354391650455908,
  'Timing Alpha IRR': -0.02722977265412274,
  'Strategy Alpha IRR': 0.010786885217840353,
  'Geography Alpha IRR': 0.001084851757246541,
  'Sizing Alpha IRR': 0.0004912907365518943,
  'Residual Alpha IRR': -0.011743565297766392})